In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [36]:
df = pd.read_csv('../day5/car evaluation/car.data', header=None)
df.rename(columns={0:'buying', 1:"maint", 2:"doors",3:"persons",4:"lug_boot",5:"safety",6:"class"}, inplace=True)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [37]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
trf = make_column_transformer((ohe, make_column_selector(dtype_include=object)),
                              remainder='passthrough', verbose_feature_names_out=False).set_output(transform='pandas')

X = df.drop('class', axis=1)
y = df['class']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=25, test_size=0.3)


In [38]:
Cs = np.linspace(0.001,5,20)
Gs = np.linspace(0.001,5,20)
dfs = ['ovo','ovr']

scores = []

for c in tqdm(Cs):
    for g in Gs:
        for d in dfs:
            pipe = Pipeline(
                [
                    ('col_trans',trf),
                    ('svm', SVC(kernel='rbf', decision_function_shape=d, gamma=g,C=c))
                ]
            )

            pipe.fit(X_train,y_train)
            y_pred = pipe.predict(X_test)
            scores.append([c,g,d,accuracy_score(y_test,y_pred)])

scores = pd.DataFrame(scores, columns=['C','Gamma','DFS','Accuracy'])
scores.sort_values('Accuracy',ascending=False)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [02:58<00:00,  8.92s/it]


,C,Gamma,DFS,Accuracy
763,5.000000,0.264105,ovr,0.980732
762,5.000000,0.264105,ovo,0.980732
723,4.736895,0.264105,ovr,0.976879
722,4.736895,0.264105,ovo,0.976879
764,5.000000,0.527211,ovo,0.976879
...,...,...,...,...
31,0.001000,3.947579,ovr,0.710983
784,5.000000,3.158263,ovo,0.710983
785,5.000000,3.158263,ovr,0.710983
786,5.000000,3.421368,ovo,0.710983
